In [1]:
!pip install -q transformers evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.2 MB/s eta 0:00:00


In [2]:
import os
import time

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from datasets import load_dataset, load_metric
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoConfig, AutoModelForQuestionAnswering, pipeline

In [7]:
squad = load_dataset("squad", split="train[:5000]")
squad = squad.train_test_split(test_size=0.2)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [5]:
squad.shape

{'train': (4000, 5), 'test': (1000, 5)}

In [6]:
sample = squad["train"][0]
print(sample)

# Каждый объект датасета содержит текст, вопрос по нему, правильный ответ и его позицию в тексте

{'id': '56d4eaca2ccc5a1400d83348', 'title': 'Beyoncé', 'context': 'Her debut single, "Crazy in Love" was named VH1\'s "Greatest Song of the 2000s", NME\'s "Best Track of the 00s" and "Pop Song of the Century", considered by Rolling Stone to be one of the 500 greatest songs of all time, earned two Grammy Awards and is one of the best-selling singles of all time at around 8 million copies. The music video for "Single Ladies (Put a Ring on It)", which achieved fame for its intricate choreography and its deployment of jazz hands, was credited by the Toronto Star as having started the "first major dance craze of both the new millennium and the Internet", triggering a number of parodies of the dance choreography and a legion of amateur imitators on YouTube. In 2013, Drake released a single titled "Girls Love Beyoncé", which featured an interpolation from Destiny Child\'s "Say My Name" and discussed his relationship with women. In January 2012, research scientist Bryan Lessard named Scaptia b

In [7]:
accuracy_score = load_metric("accuracy")


def compute_metrics(pred):
  predictions, labels = pred
  predictions = np.argmax(predictions, axis=1)
  return accuracy_score.compute(predictions=predictions, references=labels)

<ipython-input-7-175e39ac7946>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  accuracy_score = load_metric("accuracy")


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

checkpoint = "distilbert-base-uncased"
model = AutoModelForQuestionAnswering.from_pretrained(checkpoint).to(device)

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
print("Base Model: ", model.num_parameters())

Base Model:  66364418


In [8]:
sample_input_question = squad['train']['question'][101]
sample_input_context = squad['train']['context'][101]
sample_input = {"question": sample_input_question, "context": sample_input_context}

print(squad['train']['context'][101])
print(squad['train']['question'][101])
print(squad['train']['answers'][101])

Twilight Princess was released to universal critical acclaim and commercial success. It received perfect scores from major publications such as 1UP.com, Computer and Video Games, Electronic Gaming Monthly, Game Informer, GamesRadar, and GameSpy. On the review aggregators GameRankings and Metacritic, Twilight Princess has average scores of 95% and 95 for the Wii version and scores of 95% and 96 for the GameCube version. GameTrailers in their review called it one of the greatest games ever created.
What was the reception of Twilight Princess?
{'text': ['universal critical acclaim'], 'answer_start': [34]}


In [39]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
pipe = pipeline('question-answering', model=model, tokenizer=tokenizer)

In [33]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [34]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [36]:
sample_input = tokenized_squad['train'][101]
#sample_input_question = squad['train']['question'][101]

In [47]:
#WARMUP
for _ in range(10):
  _ = pipe(sample_input)

#INFERENCE
start = time.time()
for _ in range(100):
  _ = pipe(sample_input)
total_time_model = time.time() - start
infer_time = total_time_model / 100 * 1000
print(f"Avg inference time: {infer_time:.4f} ms")

Avg inference time: 230.0094 ms


In [49]:
model.save_pretrained("distilbert-base-uncased")

In [51]:
file_size_bytes = os.path.getsize('/content/distilbert-base-uncased/pytorch_model.bin')
file_size_mb = file_size_bytes / (1024 * 1024) # Перевод из байтов в мегабайты

print('model size: {:.3f}MB'.format(file_size_mb))

model size: 253.192MB


In [3]:
!python -m pip install -q optimum[onnxruntime]@git+https://github.com/huggingface/optimum.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 95.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 109.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 89.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 12.7 MB/s eta 0:00:00


### Using pipeline

In [6]:
from transformers import AutoTokenizer, pipeline
from optimum.onnxruntime import ORTModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("optimum/roberta-base-squad2")
model = ORTModelForQuestionAnswering.from_pretrained("optimum/roberta-base-squad2")
onnx_qa = pipeline("question-answering", model=model, tokenizer=tokenizer)

question, text = "Who was Jim Henson?", "Jim Henson was a nice puppet"
pred = onnx_qa(question, text)

In [13]:
inputs = tokenizer(sample_input_question, sample_input_context, return_tensors="pt")

In [15]:
#WARMUP
for _ in range(10):
  _ = model(**inputs)

#INFERENCE
start = time.time()
for _ in range(100):
  _ = model(**inputs)
total_time_model = time.time() - start
infer_time = total_time_model / 100 * 1000
print(f"Avg inference time: {infer_time:.4f} ms")

Avg inference time: 331.7231 ms


### Using ORTModelForQuestionAnswering

In [20]:
from transformers import AutoTokenizer
from optimum.onnxruntime import ORTModelForQuestionAnswering
import torch

tokenizer = AutoTokenizer.from_pretrained("optimum/roberta-base-squad2")
model = ORTModelForQuestionAnswering.from_pretrained("optimum/roberta-base-squad2")

question, text = "Who was Jim Henson?", "Jim Henson was a nice puppet"
inputs = tokenizer(sample_input_question, sample_input_context, return_tensors="pt")
start_positions = torch.tensor([1])
end_positions = torch.tensor([3])

outputs = model(**inputs, start_positions=start_positions, end_positions=end_positions)
start_scores = outputs.start_logits
end_scores = outputs.end_logits

In [22]:
#WARMUP
for _ in range(10):
  _ = model(**inputs, start_positions=start_positions, end_positions=end_positions)

#INFERENCE
start = time.time()
for _ in range(100):
  _ = model(**inputs, start_positions=start_positions, end_positions=end_positions)
total_time_model = time.time() - start
infer_time = total_time_model / 100 * 1000
print(f"Avg inference time: {infer_time:.4f} ms")

Avg inference time: 396.9597 ms


In [23]:
model.save_pretrained("optimum/roberta-base-squad2")

In [24]:
file_size_bytes = os.path.getsize('/content/optimum/roberta-base-squad2/model.onnx')
file_size_mb = file_size_bytes / (1024 * 1024) # Перевод из байтов в мегабайты

print('model size: {:.3f}MB'.format(file_size_mb))

model size: 473.314MB
